In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
Sys.time()
require( "data.table" )

# leo el dataset
#dataset <- fread("./datasets/competencia_02_crudo.csv.gz")
dataset <- fread("../../../../datasets/competencia_02_crudo.csv.gz")

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()
     

[1] "2025-11-16 21:31:12 UTC"


Loading required package: data.table
data.table 1.17.8 using 4 threads (see ?getDTthreads).  Latest news: r-datatable.com
|--------------------------------------------------|
|==================================================|


In [3]:
%%R

# Salsa Magica para 202106
dataset[, mprestamos_personales := NULL ]
dataset[, cprestamos_personales := NULL ]

         numero_de_cliente foto_mes active_quarter cliente_vip internet
                     <int>    <int>          <int>       <int>    <int>
      1:         249221323   201901              1           0        1
      2:         249227600   201901              1           0        1
      3:         249227813   201901              1           0        0
      4:         249231391   201901              1           0        1
      5:         249244739   201901              1           0        1
     ---                                                               
4717954:        1603549106   202108              0           0        0
4717955:        1603676578   202108              0           0        0
4717956:        1603770942   202108              1           0        0
4717957:        1603816886   202108              1           0        0
4717958:        1603899149   202108              0           0        0
         cliente_edad cliente_antiguedad mrentabilidad mrentabil

In [4]:
%%R
library(data.table)

# dt: data.table
# group_col: columna del mes (por defecto "foto_mes")
# cols: qué columnas revisar (por defecto, todas las numéricas)
cero_a_na_por_mes <- function(dt, group_col = "foto_mes", cols = NULL) {
  stopifnot(is.data.table(dt))
  
  if (!group_col %in% names(dt)) {
    stop(paste0("No encuentro la columna '", group_col, "' en el dataset."))
  }
  
  # Si no se especifican columnas, usamos todas las numéricas
  if (is.null(cols)) {
    cols <- names(dt)[sapply(dt, is.numeric)]
  }
  
  for (col in cols) {
    if (!col %in% names(dt)) next
    
    dt[, (col) := {
      v <- get(col)
      # si todos los valores no-NA son 0 → todo ese grupo se vuelve NA
      if (all(v == 0, na.rm = TRUE)) {
        rep(NA_real_, .N)
      } else {
        v
      }
    }, by = group_col]
  }
  
  invisible(dt)
}
# Aplicar sobre todas las columnas numéricas
cero_a_na_por_mes(dataset, group_col = "foto_mes")

In [5]:
%%R

library(data.table)

# Asegurarnos que 'dataset' es data.table
setDT(dataset)

cols <- colnames(dataset)

# =====================================================
# 1) Detectar pares m/c genéricos (no Visa/Master)
# =====================================================
m_cols_generic <- cols[grepl("^m", cols) & !grepl("^Master_|^Visa_", cols)]
c_cols_generic <- cols[grepl("^c", cols) & !grepl("^Master_|^Visa_", cols)]

pares_mc_generic <- list()

for (m_col in m_cols_generic) {
  suf <- substr(m_col, 2, nchar(m_col))   # parte después de 'm'
  c_col <- paste0("c", suf)
  if (c_col %in% c_cols_generic) {
    pares_mc_generic[[length(pares_mc_generic) + 1L]] <- list(
      tipo   = "generic",
      monto  = m_col,
      cuenta = c_col
    )
  }
}

# =====================================================
# 2) Detectar pares Visa_mXXXX / Visa_cXXXX
# =====================================================
visa_cols   <- cols[grepl("^Visa_", cols)]
visa_m_cols <- visa_cols[grepl("^Visa_m", visa_cols)]
visa_c_cols <- visa_cols[grepl("^Visa_c", visa_cols)]

pares_visa <- list()

for (m_col in visa_m_cols) {
  suf <- substr(m_col, nchar("Visa_m") + 1L, nchar(m_col))
  c_col <- paste0("Visa_c", suf)
  if (c_col %in% visa_c_cols) {
    pares_visa[[length(pares_visa) + 1L]] <- list(
      tipo   = "visa",
      monto  = m_col,
      cuenta = c_col
    )
  }
}

# =====================================================
# 3) Detectar pares Master_mXXXX / Master_cXXXX
# =====================================================
master_cols   <- cols[grepl("^Master_", cols)]
master_m_cols <- master_cols[grepl("^Master_m", master_cols)]
master_c_cols <- master_cols[grepl("^Master_c", master_cols)]

pares_master <- list()

for (m_col in master_m_cols) {
  suf <- substr(m_col, nchar("Master_m") + 1L, nchar(m_col))
  c_col <- paste0("Master_c", suf)
  if (c_col %in% master_c_cols) {
    pares_master[[length(pares_master) + 1L]] <- list(
      tipo   = "master",
      monto  = m_col,
      cuenta = c_col
    )
  }
}

# =====================================================
# 4) Unir todos los pares en un data.table + nombre ratio
# =====================================================
pares_todos <- rbindlist(
  c(pares_mc_generic, pares_visa, pares_master),
  use.names = TRUE, fill = TRUE
)

if (nrow(pares_todos) == 0L) {
  message("⚠️ No se detectaron pares m/c con el esquema definido.")
} else {
  # Agregamos el nombre de la columna ratio que se va a crear
  pares_todos[, ratio := fifelse(
    tipo == "generic",
    paste0("ratio_", substr(monto, 2, nchar(monto))),
    fifelse(
      tipo == "visa",
      paste0("ratio_Visa_", substr(monto, nchar("Visa_m") + 1L, nchar(monto))),
      paste0("ratio_Master_", substr(monto, nchar("Master_m") + 1L, nchar(monto)))
    )
  )]

  cat("✅ Pares m/c detectados y nombres de ratio:\n")
  print(pares_todos)
}

# =====================================================
# 5) Crear las columnas de ratio m/c (con NA si denom=0)
# =====================================================
nombres_ratios_creados <- character(0)

if (nrow(pares_todos) > 0L) {
  for (i in seq_len(nrow(pares_todos))) {
    tipo   <- pares_todos[i, tipo]
    m_col  <- pares_todos[i, monto]
    c_col  <- pares_todos[i, cuenta]
    ratio_nm <- pares_todos[i, ratio]

    # no pisar si ya existe
    if (ratio_nm %in% colnames(dataset)) {
      message("⏭️  Ya existe la columna ", ratio_nm, " - se saltea.")
      next
    }

    cat("➕ Creando ", ratio_nm, " = ", m_col, " / ", c_col, "\n", sep = "")

    # ⚠️ Si denominador es 0 o NA -> ratio = NA
    dataset[, (ratio_nm) := fifelse(
      !is.na(get(c_col)) & get(c_col) != 0,
      get(m_col) / get(c_col),
      NA_real_
    )]

    nombres_ratios_creados <- c(nombres_ratios_creados, ratio_nm)
  }
}

if (length(nombres_ratios_creados) > 0L) {
  cat("\n📌 Ratios creados:\n")
  print(nombres_ratios_creados)
} else {
  cat("\n⚠️ No se creó ningún ratio nuevo (ya existían o no hubo pares válidos).\n")
}

# =====================================================
# 6) Columnas de suma total de montos y de cantidades
# =====================================================
# Recalcular columnas (ahora dataset tiene ratios nuevos, pero
# como los ratios empiezan con 'ratio_' no se mezclan con m/c)
cols <- colnames(dataset)

cols_m_sum <- cols[
  grepl("^m", cols) |
    grepl("^Visa_m", cols) |
    grepl("^Master_m", cols)
]

cols_c_sum <- cols[
  grepl("^c", cols) |
    grepl("^Visa_c", cols) |
    grepl("^Master_c", cols)
]
cols_c_sum <- setdiff(cols_c_sum, "clase_ternaria")
cat("\n📦 Columnas usadas para m_total_all:\n")
print(cols_m_sum)

cat("\n📦 Columnas usadas para c_total_all:\n")
print(cols_c_sum)

if (length(cols_m_sum) > 0L) {
  dataset[, m_total_all := rowSums(.SD, na.rm = TRUE), .SDcols = cols_m_sum]
}

if (length(cols_c_sum) > 0L) {
  dataset[, c_total_all := rowSums(.SD, na.rm = TRUE), .SDcols = cols_c_sum]
}

cat("\n✅ m_total_all y c_total_all creadas.\n")


✅ Pares m/c detectados y nombres de ratio:
       tipo                           monto                          cuenta
     <char>                          <char>                          <char>
 1: generic               mcuenta_corriente               ccuenta_corriente
 2: generic                    mcaja_ahorro                    ccaja_ahorro
 3: generic           mprestamos_prendarios           cprestamos_prendarios
 4: generic         mprestamos_hipotecarios         cprestamos_hipotecarios
 5: generic                     minversion2                     cinversion2
 6: generic     mcuenta_debitos_automaticos     ccuenta_debitos_automaticos
 7: generic                mpagodeservicios                cpagodeservicios
 8: generic                 mpagomiscuentas                 cpagomiscuentas
 9: generic     mcajeros_propios_descuentos     ccajeros_propios_descuentos
10: generic        mtarjeta_visa_descuentos        ctarjeta_visa_descuentos
11: generic      mtarjeta_master_descuentos  

In [6]:
%%R
# el mes 1,2, ..12 , podria servir para detectar estacionalidad
dataset[, kmes := foto_mes %% 100]

# creo un ctr_quarter que tenga en cuenta cuando
# los clientes hace 3 menos meses que estan
# ya que seria injusto considerar las transacciones medidas en menor tiempo
dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]
dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5.0]
dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2.0]
dataset[cliente_antiguedad == 3, ctrx_quarter_normalizado := ctrx_quarter * 1.2]

# variable extraida de una tesis de maestria de Irlanda, se perdió el link
dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

Sys.time()

[1] "2025-11-16 21:33:35 UTC"


In [7]:
%%R
if( !require("Rcpp")) install.packages("Rcpp", repos = "http://cran.us.r-project.org")
require("Rcpp")

Loading required package: Rcpp


In [8]:
%%R
# se calculan para los 6 meses previos el minimo, maximo y
#  tendencia calculada con cuadrados minimos
# la formula de calculo de la tendencia puede verse en
#  https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
# para la maxíma velocidad esta funcion esta escrita en lenguaje C,
# y no en la porqueria de R o Python

cppFunction("NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde )
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) )
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h < libre; h++ ) {
        if( y[h] < vmin ) vmin = y[h];
        if( y[h] > vmax ) vmax = y[h];
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ;
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}")
     

In [9]:
%%R
# calcula la tendencia de las variables cols de los ultimos 6 meses
# la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
# La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas <- function(
    dataset, cols, ventana = 6, tendencia = TRUE,
    minimo = TRUE, maximo = TRUE, promedio = TRUE,
    ratioavg = FALSE, ratiomax = FALSE) {
  gc(verbose= FALSE)
  # Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion <- ventana

  last <- nrow(dataset)

  # creo el vector_desde que indica cada ventana
  # de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids <- dataset[ , numero_de_cliente ]

  vector_desde <- seq(
    -ventana_regresion + 2,
    nrow(dataset) - ventana_regresion + 1
  )

  vector_desde[1:ventana_regresion] <- 1

  for (i in 2:last) {
    if (vector_ids[i - 1] != vector_ids[i]) {
      vector_desde[i] <- i
    }
  }
  for (i in 2:last) {
    if (vector_desde[i] < vector_desde[i - 1]) {
      vector_desde[i] <- vector_desde[i - 1]
    }
  }

  for (campo in cols) {
    nueva_col <- fhistC(dataset[, get(campo)], vector_desde)

    if (tendencia) {
      dataset[, paste0(campo, "_tend", ventana) :=
        nueva_col[(0 * last + 1):(1 * last)]]
    }

    if (minimo) {
      dataset[, paste0(campo, "_min", ventana) :=
        nueva_col[(1 * last + 1):(2 * last)]]
    }

    if (maximo) {
      dataset[, paste0(campo, "_max", ventana) :=
        nueva_col[(2 * last + 1):(3 * last)]]
    }

    if (promedio) {
      dataset[, paste0(campo, "_avg", ventana) :=
        nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratioavg) {
      dataset[, paste0(campo, "_ratioavg", ventana) :=
        get(campo) / nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratiomax) {
      dataset[, paste0(campo, "_ratiomax", ventana) :=
        get(campo) / nueva_col[(2 * last + 1):(3 * last)]]
    }
  }
}
     

In [10]:
%%R

# Feature Engineering Historico
# Creacion de LAGs
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
  colnames(dataset),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
))

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
  paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 2
dataset[,
  paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
  dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
  dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}

Sys.time()
     

[1] "2025-11-16 21:35:40 UTC"


In [11]:
%%R
#parámetros de Feature Engineering Histórico de Tendencias

PARAM <- list()
PARAM$experimento <- "apo-001" ########## CAMBIA CAMBIA
PARAM$semilla_primigenia <- 102191 ######### CAMBIA CAMBIA
PARAM$FE_histTendencias$run       <- TRUE
PARAM$FE_histTendencias$ventana   <- 6

PARAM$FE_histTendencias$tendencia <- TRUE
PARAM$FE_histTendencias$minimo    <- FALSE
PARAM$FE_histTendencias$maximo    <- FALSE
PARAM$FE_histTendencias$promedio  <- FALSE
PARAM$FE_histTendencias$ratioavg  <- FALSE
PARAM$FE_histTendencias$ratiomax  <- FALSE


In [12]:
%%R

cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_histTendencias$run) {
    TendenciaYmuchomas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_histTendencias$ventana,
    tendencia = PARAM$FE_histTendencias$tendencia,
    minimo = PARAM$FE_histTendencias$minimo, 
    maximo = PARAM$FE_histTendencias$maximo, 
    promedio = PARAM$FE_histTendencias$promedio,
    ratioavg = PARAM$FE_histTendencias$ratioavg,
    ratiomax = PARAM$FE_histTendencias$ratiomax
  )
}

ncol(dataset)
Sys.time()

[1] "2025-11-16 21:37:00 UTC"


In [13]:
%%R

if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")
     

Loading required package: lightgbm


In [14]:
%%R
AgregaVarRandomForest <- function() {

  cat("inicio AgregaVarRandomForest()\n")
  gc(verbose = FALSE)

  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% c("BAJA+2", "BAJA+1"), clase01 := 1L ]

  campos_buenos <- setdiff(colnames(dataset),
                           c("clase_ternaria", "clase01"))

  dataset[, entrenamiento := as.integer(foto_mes %in% PARAM$train$training)]

  dtrain <- lgb.Dataset(
    data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    free_raw_data = FALSE
  )

  modelo <- lgb.train(
    data = dtrain,
    param = PARAM$lgb_param,
    verbose = -100
  )

  cat("Fin construccion RandomForest\n")

  lgb.save(modelo, file = "modelo.model")

  qarbolitos <- copy(PARAM$lgb_param$num_iterations)

  periodos <- dataset[, unique(foto_mes)]

  for (periodo in periodos) {

    cat("periodo = ", periodo, "\n")

    datamatrix <- data.matrix(
      dataset[foto_mes == periodo, campos_buenos, with = FALSE]
    )

    cat("Inicio prediccion\n")
    prediccion <- predict(modelo, datamatrix, type = "leaf")
    cat("Fin prediccion\n")

    for (arbolito in 1:qarbolitos) {

      cat(arbolito, " ")

      hojas_arbol <- unique(prediccion[, arbolito])

      for (pos in seq_along(hojas_arbol)) {

        nodo_id <- hojas_arbol[pos]

        dataset[foto_mes == periodo,
          paste0("rf_", sprintf("%03d", arbolito), "_", sprintf("%03d", nodo_id))
        := as.integer(nodo_id == prediccion[, arbolito])]

      }

      rm(hojas_arbol)
    }

    cat("\n")
    rm(prediccion)
    rm(datamatrix)
    gc(verbose = FALSE)
  }

  gc(verbose = FALSE)

  dataset[, clase01 := NULL ]
}


In [ ]:
%%R

# Crear lista PARAM si no existe
if (!exists("PARAM")) PARAM <- list()

# Parametros de Feature Engineering a partir de hojas de Random Forest

# Estos CUATRO parametros son los que se deben modificar
PARAM$arbolitos       <- 20
PARAM$hojas_por_arbol <- 16
PARAM$datos_por_hoja  <- 100
PARAM$mtry_ratio      <- 0.2

# Estos son quasi fijos
PARAM$train <- list()
PARAM$train$training <- c(202101, 202102, 202103)

# Estos TAMBIEN son quasi fijos
PARAM$lgb_param <- list(
  
  # parámetros que se pueden cambiar
  num_iterations        = PARAM$arbolitos,
  num_leaves            = PARAM$hojas_por_arbol,
  min_data_in_leaf      = PARAM$datos_por_hoja,
  feature_fraction_bynode = PARAM$mtry_ratio,

  # para que LightGBM emule Random Forest
  boosting        = "rf",
  bagging_fraction = (1.0 - 1.0/exp(1.0)),
  bagging_freq     = 1.0,
  feature_fraction = 1.0,

  # genéricos de LightGBM
  max_bin             = 31L,
  objective           = "binary",
  first_metric_only   = TRUE,
  boost_from_average  = TRUE,
  feature_pre_filter  = FALSE,
  force_row_wise      = TRUE,
  verbosity           = -100,
  max_depth           = -1L,
  min_gain_to_split   = 0.0,
  min_sum_hessian_in_leaf = 0.001,
  lambda_l1           = 0.0,
  lambda_l2           = 0.0,

  pos_bagging_fraction = 1.0,
  neg_bagging_fraction = 1.0,
  is_unbalance         = FALSE,
  scale_pos_weight     = 1.0,

  drop_rate = 0.1,
  max_drop  = 50,
  skip_drop = 0.5,

  extra_trees = FALSE
)


In [16]:
%%R
AgregaVarRandomForest()

inicio AgregaVarRandomForest()
Fin construccion RandomForest
periodo =  201901 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201902 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201903 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201904 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201905 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201906 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201907 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201908 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)




      2:                            NA                  NA                   NA
      3:                             1            -6841.62                    1
      4:                             1            -7017.28                    1
      5:                             1            -6595.66                    1
     ---                                                                       
4717954:                            NA                  NA                   NA
4717955:                            NA                  NA                   NA
4717956:                            NA                  NA                   NA
4717957:                            NA                  NA                   NA
4717958:                            NA                  NA                   NA
         ctarjeta_debito_transacciones_lag2 mautoservicio_lag2
                                      <int>              <num>
      1:                                 NA                 NA
      2:  

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)




      5:                     NA                         0
     ---                                                 
4717954:                     NA                        NA
4717955:                     NA                        NA
4717956:                     NA                        NA
4717957:                     NA                        NA
4717958:                     NA                        NA
         Master_delinquency_delta2 Master_status_delta1 Master_status_delta2
                             <int>                <int>                <int>
      1:                        NA                   NA                   NA
      2:                        NA                    0                   NA
      3:                         0                    0                    0
      4:                         0                    0                    0
      5:                         0                    0                    0
     ---                                              

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



          0          0
      2:          0          0          0          0          1          0
      3:          0          0          0          0          1          0
      4:          0          0          0          0          1          0
      5:          0          0          0          0          1          0
     ---                                                                  
4717954:          0          0          0          1          0          0
4717955:          0          0          0          0          0          1
4717956:          0          0          0          0          0          0
4717957:          0          0          0          0          0          0
4717958:          1          0          0          0          0          0
         rf_008_001 rf_009_004 rf_009_014 rf_009_001 rf_009_015 rf_010_008
              <int>      <int>      <int>      <int>      <int>      <int>
      1:          0          1          0          0          0          1
  

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
%%R -o dataset_final
# asegurar que existe 'dataset'
dataset_final <- as.data.frame(dataset)  # forzar data.frame


In [ ]:
import polars as pl

# dataset_final ahora es un pandas.DataFrame traido desde R
df_pl = pl.from_pandas(dataset_final)

df_pl


numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,tcuentas,ccuenta_corriente,mcuenta_corriente_adicional,mcuenta_corriente,ccaja_ahorro,mcaja_ahorro,mcaja_ahorro_adicional,mcaja_ahorro_dolares,cdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,ctarjeta_master,ctarjeta_master_transacciones,mtarjeta_master_consumo,cprestamos_prendarios,mprestamos_prendarios,cprestamos_hipotecarios,mprestamos_hipotecarios,cplazo_fijo,…,rf_016_014,rf_016_009,rf_016_004,rf_016_011,rf_016_013,rf_016_000,rf_016_010,rf_017_009,rf_017_002,rf_017_000,rf_017_010,rf_017_008,rf_017_004,rf_017_015,rf_017_011,rf_017_014,rf_017_007,rf_017_012,rf_018_013,rf_018_001,rf_018_000,rf_018_007,rf_018_006,rf_019_014,rf_019_015,rf_019_010,rf_019_008,rf_019_000,rf_019_012,rf_019_013,rf_020_001,rf_020_004,rf_020_014,rf_020_010,rf_018_011,rf_006_006,rf_019_009
i32,i32,i32,i32,i32,i32,i32,f64,f64,f64,f64,f64,i32,i32,i32,f64,f64,i32,f64,f64,f64,i32,f64,i32,i32,f64,i32,i32,f64,i32,i32,f64,i32,f64,i32,f64,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
249221323,201901,1,0,1,44,69,86.33,14610.01,625.85,-326.52,-133.43,11,1,1,0.0,-4649.92,2,2109.53,0.0,0.0,1,-6841.62,1,0,0.0,1,10,11330.95,1,14,13379.13,0,0.0,0,0.0,0,…,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648
249221323,201902,1,0,1,44,70,428.26,13467.47,438.31,-13.86,5.29,11,1,1,0.0,-6303.71,2,0.0,0.0,0.0,1,-7017.28,1,0,0.0,1,9,7386.79,1,25,30422.11,0,0.0,0,0.0,0,…,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648
249221323,201903,1,0,1,44,71,1165.24,12321.2,530.21,59.44,481.93,11,1,1,0.0,-6399.17,2,0.0,0.0,0.0,1,-6595.66,1,0,0.0,1,15,32690.73,1,15,15178.88,0,0.0,0,0.0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-2147483648,-2147483648,-2147483648
249221323,201904,1,0,1,44,72,3305.15,14492.34,581.15,910.65,1411.6,11,1,1,0.0,-2131.01,2,538.59,0.0,0.0,1,-461.22,1,2,2580.6,1,17,19501.62,1,18,17135.17,0,0.0,0,0.0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-2147483648,-2147483648
249221323,201905,1,0,1,44,73,null,null,null,null,null,11,1,1,0.0,-2633.2,2,0.0,0.0,0.0,1,-3558.82,1,0,0.0,1,14,16886.74,1,12,15075.0,0,0.0,0,0.0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-2147483648,0,-2147483648,-2147483648,-2147483648,-2147483648,0,0,-2147483648,-2147483648,0,-2147483648,-2147483648
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1603549106,202108,0,0,0,44,1,94.13,94.13,0.0,0.0,80.25,7,1,1,0.0,0.0,2,5862.46,0.0,0.0,1,90868.39,1,0,0.0,1,0,0.0,1,0,0.0,0,0.0,0,0.0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-2147483648,-2147483648
1603676578,202108,0,0,0,56,1,2705.4,2705.4,2698.92,5.53,0.0,7,1,1,0.0,-236.14,2,0.0,0.0,0.0,1,-3660.47,1,0,0.0,1,0,0.0,1,0,0.0,0,0.0,0,0.0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [19]:
df_pl.write_parquet("../../../../Desktop/dataset_benja_421.parquet")

In [20]:

import numpy as np
import lightgbm as lgb 
from typing import Tuple
from datetime import datetime
from dateutil.relativedelta import relativedelta
import polars as pl
import gzip


In [ ]:
def read_polar(path):
   
    #path = r"G:\My Drive\dmeyf\competencia_02_crudo.csv.gz"

    # Leer el CSV comprimido
    dataset = pl.read_csv(
        path,
        separator=",",            
        has_header=True,    
        infer_schema_length=5000,  
        ignore_errors=True, 
        low_memory=True     
    )

    #print(dataset.shape)
    #print(dataset.head())
    return dataset

def write_polar_gz(dataset,path_target):
    with gzip.open(path_target, "wt", encoding="utf-8") as f:
        dataset.write_csv(f)
    return


def generar_columna_target(dataset):
    # target = 1 si clase_ternaria  ["BAJA+1", "BAJA+2"], sino 0
    dataset = dataset.with_columns([
        pl.col("clase_ternaria")
          .is_in(["BAJA+1", "BAJA+2"])
          .cast(pl.Int8)
          .alias("target"),
    
        # clase_peso = 1.0
        pl.lit(1.0).alias("clase_peso")
    ])
    return dataset

def undersampling_experimento(dataset,undersampling,train_months,seed):
    # --- parámetros de entrenamiento ---  
    dataset = dataset.filter(pl.col("foto_mes").is_in(train_months))

    # --- generar la columna azar reproducible ---
    rng = np.random.default_rng(seed)
    dataset = dataset.with_columns(
        pl.Series("azar", rng.random(len(dataset)))  # uniforme entre 0 y 1
    )
    
    # --- inicializar columna training ---
    dataset = dataset.with_columns(pl.lit(0).alias("undersampling"))
    
    # --- aplicar la misma condición que en R ---
    dataset = dataset.with_columns(
        pl.when(
                (pl.col("azar") <= undersampling) |
                (pl.col("clase_ternaria").is_in(["BAJA+1", "BAJA+2"]))
        )
        .then(1)
        .otherwise(0)
        .alias("undersampling")
    )
    
    # --- chequeo ---
    #print(dataset.select(pl.col("undersampling")).to_series().value_counts())
    #print(dataset.group_by(["undersampling", "clase_ternaria"]).len().sort("undersampling"))
    return dataset


# ============================================================
# Función de ganancia (métrica personalizada para LightGBM)
# ============================================================

def lgb_gan_eval(y_pred, data):
    """
    Métrica personalizada de ganancia para LightGBM.
    Usa la misma lógica que 'calcular_ganancia_optima', 
    pero adaptada al formato feval de LightGBM.
    """
    y_true = data.get_label()

    # Rango de envíos a evaluar
    envios_min, envios_max = 8000, 13000
    step = 250

    # Ordenar predicciones descendente
    idx_ordenados = np.argsort(y_pred)[::-1]
    y_true_ordenado = y_true[idx_ordenados]

    mejor_ganancia = -np.inf
    for n_envios in range(envios_min, envios_max + 1, step):
        predichos_como_baja = y_true_ordenado[:n_envios]
        vp = np.sum(predichos_como_baja == 1)     # verdaderos positivos
        fp = n_envios - vp                        # falsos positivos
        ganancia = vp * 780000 - fp * 20000
        if ganancia > mejor_ganancia:
            mejor_ganancia = ganancia

    # True = maximizar
    return 'gan_eval', mejor_ganancia, True


# ============================================================
# Funcion auxiliar de validación externa
# ============================================================

def calcular_ganancia_optima(y_true: np.ndarray, y_pred_proba: np.ndarray,envios_min: int = 8000, envios_max: int = 13000) -> Tuple[float, int]:
    """
    Encuentra el número óptimo de envíos que maximiza la ganancia
    (reutilizable fuera del feval).
    """
    idx_ordenados = np.argsort(y_pred_proba)[::-1]
    y_true_ordenado = y_true[idx_ordenados]

    mejor_ganancia = -np.inf
    mejor_n_envios = envios_min
    step = 250

    for n_envios in range(envios_min, envios_max + 1, step):
        predichos_como_baja = y_true_ordenado[:n_envios]
        vp = np.sum(predichos_como_baja == 1)
        fp = n_envios - vp
        ganancia = vp * 780000 - fp * 20000
        if ganancia > mejor_ganancia:
            mejor_ganancia = ganancia
            mejor_n_envios = n_envios

    return mejor_ganancia, mejor_n_envios

def generar_nulos(df):
    #para ver si va perdiendo productos
    df = df.with_columns(
        pl.sum_horizontal([pl.col(c).is_null().cast(pl.Int64) for c in df.columns])
        .alias("cant_nulos")
    )
    return df

def generar_aguinaldo(df):
    df = df.with_columns(
        pl.when(pl.col("kmes").is_in([6, 12]))
        .then(1)
        .otherwise(0)
        .alias("aguinaldo")
    )
    return df



def procesamiento_general(df: pl.DataFrame):
    """
    Recibe un Polars DataFrame ya featureado (por R) y aplica
    los ultimos pasos de FE en Python.
    """

    print("Dataset featureado recibido desde memoria (R -> Python -> Polars)")
    print(f"Filas: {df.height:,}  |  Columnas: {df.width}")

    # meses disponibles
    meses = df["foto_mes"].unique().sort().to_list()
    print(f"Meses en el dataset: {meses}")

    print("Generando cant_nulos...")
    df = generar_nulos(df)
    print("cant_nulos OK")

    print("Generando aguinaldo...")
    df = generar_aguinaldo(df)
    print("aguinaldo OK")

    print("Generando target...")
    df = generar_columna_target(df)
    print("Target creado")

    return df, meses
def ultimos_12_meses_y_prev(month, months_all):
    # convertir el entero AAAAMM a datetime
    year = month // 100
    m = month % 100
    fecha = datetime(year, m, 1)

    # mes anterior
    fecha_prev = fecha - relativedelta(months=1)
    prev = fecha_prev.year * 100 + fecha_prev.month

    # rango de ultimos 12 meses (hasta el mes anterior)
    fecha_inicio = fecha - relativedelta(months=13)
    start = fecha_inicio.year * 100 + fecha_inicio.month
    end = prev  # no incluye el mes que pasas

    ultimos_12 = [m for m in months_all if start <= m <= end]

    return prev, ultimos_12

def ultimos_12_meses_hasta(month_hasta, months_all):
    """
    Devuelve los últimos 12 meses HASTA el mes especificado (inclusive)
    Por ejemplo, si month_hasta=202104, devuelve los 12 meses hasta 202104
    """
    # convertir el entero AAAAMM a datetime
    year = month_hasta // 100
    m = month_hasta % 100
    fecha_hasta = datetime(year, m, 1)
    
    # 11 meses antes (para tener 12 meses en total incluyendo month_hasta)
    fecha_inicio = fecha_hasta - relativedelta(months=11)
    start = fecha_inicio.year * 100 + fecha_inicio.month
    
    # Filtrar meses en el rango
    meses_train = [mes for mes in months_all if start <= mes <= month_hasta]
    
    return meses_train



In [ ]:
def preparar_data(df, TRAIN_MONTHS, VAL_MONTH, UNDERSAMPLING, n_canaritos):
    print("="*70)
    print("PREPARACIÓN DE DATOS")
    print("="*70)
    df_completo = df.clone()
    
    dataset_train_procesado = undersampling_experimento(
        df_completo, 
        UNDERSAMPLING, 
        TRAIN_MONTHS, 
        SEED_UNDERSAMPLE
    )
    print("undersampling ok")
    
    print(f"Separando train (meses: {TRAIN_MONTHS}) y validación (mes: {VAL_MONTH})...")
    dataset_train = dataset_train_procesado.filter(
        (pl.col("foto_mes").is_in(TRAIN_MONTHS)) & 
        (pl.col("undersampling") == 1)
    )
    dataset_val = df_completo.filter(pl.col("foto_mes") == VAL_MONTH)
    
    print(f"   Train: {len(dataset_train)} registros")
    print(f"   Val: {len(dataset_val)} registros")
    
    print("Creando target específico para validación (solo BAJA+2 = 1)...")
    dataset_val = dataset_val.with_columns(
        (pl.col('clase_ternaria') == 'BAJA+2').cast(pl.Int8).alias('target_baja2')
    )
    
    # Generar canaritos (deben ser las PRIMERAS columnas)
    print(f"Generando {n_canaritos} canaritos...")
    
    # Crear canaritos para train
    np.random.seed(42)
    canaritos_train = np.random.uniform(-1, 1, size=(len(dataset_train), n_canaritos))
    
    # Crear canaritos para val
    canaritos_val = np.random.uniform(-1, 1, size=(len(dataset_val), n_canaritos))
    
    print("Preparando matrices numpy...")
    
    exclude_cols = ['clase_ternaria', 'target', 'target_baja2', 'foto_mes', 
                    'numero_de_cliente', 'periodo0', 'periodo1', 'periodo2', 
                    'azar', 'undersampling', 'clase_peso', 'mprestamos_personales', 
                    'cprestamos_personales']
    
    feature_cols = [col for col in dataset_train.columns if col not in exclude_cols]
    
    print(f"   Número de features: {len(feature_cols)}")
    
    # Prints de debug para identificar dónde se traba
    print("   📊 Extrayendo targets...")
    y_train = dataset_train['target'].to_numpy()
    y_val = dataset_val['target_baja2'].to_numpy()
    print("   ✅ Targets extraídos")
    
    print("   📊 Convirtiendo features de TRAIN a numpy... (esto puede tardar)")
    X_train_features = dataset_train.select(feature_cols).to_numpy()
    print(f"   ✅ Train convertido: shape {X_train_features.shape}")
    
    print("   📊 Convirtiendo features de VAL a numpy... (esto puede tardar)")
    X_val_features = dataset_val.select(feature_cols).to_numpy()
    print(f"   ✅ Val convertido: shape {X_val_features.shape}")
    
    # Concatenar canaritos AL PRINCIPIO (mandatorio)
    print("   📊 Concatenando canaritos al principio...")
    X_train = np.concatenate([canaritos_train, X_train_features], axis=1)
    X_val = np.concatenate([canaritos_val, X_val_features], axis=1)
    print("   ✅ Canaritos concatenados")
    
    print(f"\n   ✅ X_train shape: {X_train.shape} (incluye {n_canaritos} canaritos)")
    print(f"   ✅ y_train shape: {y_train.shape}")
    print(f"   ✅ X_val shape: {X_val.shape} (incluye {n_canaritos} canaritos)")
    print(f"   ✅ y_val shape: {y_val.shape}")
    
    return X_train, y_train, X_val, y_val
def entrenar_zlightgbm(X_train: np.ndarray, y_train: np.ndarray,
                       X_future: np.ndarray, y_future: np.ndarray,
                       canaritos: int, gradient_bound: float,
                       multi_seed: bool = True):
    """
    Entrena usando zLightGBM con los hiperparámetros elegidos
    y usa FUTURE (202106) solo para evaluar ganancia.
    """
    print("="*70)
    print("ENTRENAMIENTO CON zLightGBM")
    print("="*70)
    print(f"Train set: {X_train.shape[0]} registros")
    print(f"  - BAJA+2 (target=1): {np.sum(y_train == 1)} ({np.mean(y_train)*100:.2f}%)")
    print(f"  - Otros (target=0): {np.sum(y_train == 0)} ({(1-np.mean(y_train))*100:.2f}%)")
    print(f"\nFuture set: {X_future.shape[0]} registros (p.ej. 202106)")
    print(f"  - BAJA+2 (target=1): {np.sum(y_future == 1)} ({np.mean(y_future)*100:.2f}%)")
    print(f"  - Otros (target=0): {np.sum(y_future == 0)} ({(1-np.mean(y_future))*100:.2f}%)")
    print(f"\nNúmero de features: {X_train.shape[1]}")
    
    # Parametros base para zLightGBM (version "apo")
    params = {
        "boosting_type": "gbdt",
        "objective": "binary",
        "metric": "custom",
        "first_metric_only": True,
        "boost_from_average": True,
        "feature_pre_filter": False,
        "verbosity": -1,
        "force_row_wise": True,
        
        # Balanceo
        "is_unbalance": False,
        "scale_pos_weight": 1.0,
        
        # Hiperparametros de zLightGBM
        "canaritos": canaritos,          # 5
        "gradient_bound": gradient_bound,  # 0.4
        
        # Hiperparametros principales 
        "num_iterations": 1000,          
        "learning_rate": 1.0,
        "feature_fraction": 0.7,
        "num_leaves": 64,                
        "min_data_in_leaf": 200,
        "bagging_fraction": 0.8,
        "bagging_freq": 5,
        "lambda_l1": 0.0,
        "lambda_l2": 0.0,
    }
    
    # Multi-seed
    if multi_seed:
        # ahora usamos 20 semillas (semillerio = 20)
        semillas = [
            276779, 138349, 554179, 501997, 465077,
            323467, 262489, 237071, 812807, 149011,
            145897, 247991, 457117, 481051, 962789,
            140689, 423439, 808627, 460373, 861739
        ]
        print(f"\nUsando MULTISEED con {len(semillas)} semillas")
    else:
        semillas = [276779]
        print(f"\nUsando una sola semilla")
    
    ganancias = []
    n_envios_list = []
    modelos = []
    
    for i, seed in enumerate(semillas, 1):
        print(f"\n{'─'*70}")
        print(f"Entrenando con semilla {i}/{len(semillas)}: {seed}")
        print(f"{'─'*70}")
        
        params_con_seed = params.copy()
        params_con_seed["seed"] = seed
        params_con_seed["bagging_seed"] = seed
        params_con_seed["feature_fraction_seed"] = seed
        
        train_data = lgb.Dataset(X_train, label=y_train)
        
        # Entrenamos solo con train, FUTURE no entra acá
        model = lgb.train(
            params_con_seed,
            train_data,
            feval=lgb_gan_eval,
            callbacks=[

                lgb.log_evaluation(1)  # log de gan_eval en cada árbol
            ]
        )
        
        # Predicciones y ganancia en FUTURE
        y_pred_proba = model.predict(X_future, num_iteration=model.best_iteration)
        ganancia, n_envios = calcular_ganancia_optima(y_future, y_pred_proba)
        
        ganancias.append(ganancia)
        n_envios_list.append(n_envios)
        modelos.append(model)
        
        print(f"✅ Ganancia FUTURE: ${ganancia:,.0f}")
        print(f"   Envíos óptimos FUTURE: {n_envios}")
    
    ganancia_prom = np.mean(ganancias)
    n_envios_prom = np.mean(n_envios_list)
    

    print(f"🎉 ENTRENAMIENTO COMPLETADO (FUTURE = 202106)")
    print(f"{'='*70}")
    print(f"\nGanancia promedio FUTURE: ${ganancia_prom:,.0f}")
    print(f"Ganancia mínima FUTURE:   ${np.min(ganancias):,.0f}")
    print(f"Ganancia máxima FUTURE:   ${np.max(ganancias):,.0f}")
    print(f"Desviación estándar:      ${np.std(ganancias):,.0f}")
    print(f"\nEnvíos promedio FUTURE:   {n_envios_prom:.0f}")
    print(f"{'='*70}\n")
    
    return modelos, ganancia_prom, n_envios_prom


In [ ]:
def enn_polars(dataset, k_neighbors=7, n_jobs=-1):
    """
    Edited Nearest Neighbors (ENN) usando Polars.
    Elimina ejemplos de la clase mayoritaria mal clasificados por sus vecinos.
    
    Parámetros:
    -----------
    dataset : pl.DataFrame
        Dataset con columna 'undersampling' = 1 (registros a procesar)
    k_neighbors : int
        Número de vecinos más cercanos a considerar
    n_jobs : int
        Número de cores para paralelizar (-1 = todos)
    
    Retorna:
    --------
    pl.DataFrame con nueva columna 'enn_keep' (1=mantener, 0=eliminar)
    """
    import time
    from sklearn.neighbors import NearestNeighbors
    
    print(f"\n{'='*70}")
    print(f"APLICANDO ENN (k={k_neighbors})")
    print(f"{'='*70}")
    
    inicio = time.time()
    
    # FILTRAR: Solo procesar registros con undersampling=1
    print("\n1. Filtrando registros con undersampling=1...")
    datos_procesables = dataset.filter(pl.col("undersampling") == 1)
    print(f"   Total a procesar: {len(datos_procesables):,} registros")
    
    # SEPARAR por clase
    print("\n2. Separando por clase...")
    continua = datos_procesables.filter(pl.col("clase_ternaria") == "CONTINUA")
    bajas = datos_procesables.filter(
        pl.col("clase_ternaria").is_in(["BAJA+1", "BAJA+2"])
    )
    
    print(f"   CONTINUA: {len(continua):,}")
    print(f"   BAJA+1/2: {len(bajas):,}")
    
    # PREPARAR features numéricas (sin columnas meta)
    print("\n3. Preparando features numéricas...")
    exclude_cols = {
        "numero_de_cliente", "foto_mes", "clase_ternaria", "target", 
        "clase_peso", "periodo0", "periodo1", "periodo2", 
        "azar", "undersampling"
    }
    
    numeric_types = (
        pl.Int8, pl.Int16, pl.Int32, pl.Int64,
        pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
        pl.Float32, pl.Float64
    )
    
    feature_cols = [
        c for c, dtype in zip(datos_procesables.columns, datos_procesables.dtypes)
        if c not in exclude_cols and isinstance(dtype, numeric_types)
    ]
    
    print(f"   Features seleccionadas: {len(feature_cols)}")
    
    # 4️ CONVERTIR a numpy (más rápido para scikit-learn)
    print("\n4. Convirtiendo a numpy...")
    X_continua = continua.select(feature_cols).fill_null(0).fill_nan(0).to_numpy()
    X_bajas = bajas.select(feature_cols).fill_null(0).fill_nan(0).to_numpy()
    
    # ENTRENAR k-NN con TODAS las instancias (CONTINUA + BAJAS)
    print(f"\n5. Entrenando k-NN (k={k_neighbors})...")
    print("   Esto puede tomar varios minutos...")
    
    tiempo_knn_inicio = time.time()
    
    # Combinar todos los datos para entrenar el modelo
    X_todos = np.vstack([X_continua, X_bajas])
    y_todos = np.concatenate([
        np.zeros(len(X_continua)),  # CONTINUA = 0
        np.ones(len(X_bajas))        # BAJAS = 1
    ])
    
    # Entrenar k-NN
    knn = NearestNeighbors(n_neighbors=k_neighbors, n_jobs=n_jobs, algorithm='auto')
    knn.fit(X_todos)
    
    tiempo_knn = time.time() - tiempo_knn_inicio
    print(f"   ✓ k-NN entrenado en {tiempo_knn:.2f} segundos")
    
    # PREDECIR vecinos para CONTINUA (solo evaluamos la clase mayoritaria)
    print(f"\n6. Buscando {k_neighbors} vecinos más cercanos para cada CONTINUA...")
    print(f"   Procesando {len(X_continua):,} registros...")
    
    tiempo_vecinos_inicio = time.time()
    
    # Encontrar vecinos
    distances, indices = knn.kneighbors(X_continua)
    
    tiempo_vecinos = time.time() - tiempo_vecinos_inicio
    print(f"   ✓ Vecinos encontrados en {tiempo_vecinos:.2f} segundos")
    
    # DECIDIR que registros CONTINUA mantener
    print("\n7. Evaluando regla ENN...")
    print("   Regla: Mantener CONTINUA si mayoría de vecinos son CONTINUA")
    
    # Para cada registro CONTINUA, verificar si la mayoría de vecinos son CONTINUA (clase 0)
    vecinos_clases = y_todos[indices]  # shape: (n_continua, k_neighbors)
    
    # Contar cuántos vecinos son CONTINUA (clase 0)
    votos_continua = np.sum(vecinos_clases == 0, axis=1)
    
    # Mantener si mayoría son CONTINUA (> k/2)
    mantener_continua = votos_continua > (k_neighbors / 2)
    
    eliminados = np.sum(~mantener_continua)
    print(f"   Registros CONTINUA a eliminar: {eliminados:,} ({eliminados/len(continua)*100:.2f}%)")
    print(f"   Registros CONTINUA a mantener: {np.sum(mantener_continua):,} ({np.sum(mantener_continua)/len(continua)*100:.2f}%)")
    
    #  CREAR columna enn_keep en dataset original
    print("\n8. Creando columna 'enn_keep' en dataset...")
    
    # Inicializar: todos con undersampling=0 se eliminan (enn_keep=0)
    dataset = dataset.with_columns(
        pl.when(pl.col("undersampling") == 0)
        .then(0)
        .otherwise(1)  # Por defecto, mantener los que pasaron undersampling
        .alias("enn_keep")
    )
    
    # Obtener IDs de CONTINUA a eliminar
    ids_continua = continua.select("numero_de_cliente", "foto_mes").to_numpy()
    ids_eliminar_continua = ids_continua[~mantener_continua]
    
    # Convertir a set de tuplas para búsqueda rápida
    eliminar_set = set(map(tuple, ids_eliminar_continua))
    
    # Marcar registros CONTINUA a eliminar
    dataset = dataset.with_columns(
        pl.when(
            (pl.col("clase_ternaria") == "CONTINUA") &
            pl.struct(["numero_de_cliente", "foto_mes"]).map_elements(
                lambda x: (x["numero_de_cliente"], x["foto_mes"]) in eliminar_set,
                return_dtype=pl.Boolean
            )
        )
        .then(0)
        .otherwise(pl.col("enn_keep"))
        .alias("enn_keep")
    )
    
    tiempo_total = time.time() - inicio
    
    # 9️ RESUMEN FINAL
    print(f"\n{'='*70}")
    print("RESUMEN ENN")
    print(f"{'='*70}")
    print(dataset.group_by(["enn_keep", "clase_ternaria"]).len().sort("enn_keep"))
    print(f"\nTiempo total ENN: {tiempo_total:.2f} segundos ({tiempo_total/60:.2f} minutos)")
    print(f"{'='*70}\n")
    
    return dataset

In [24]:
def detectar_inf_polars(df):
    """
    Detecta columnas que contienen valores +inf o -inf en un Polars DataFrame.
    Devuelve un dict: {columna: cantidad_inf}
    """
    inf_dict = {}

    for c, dt in zip(df.columns, df.dtypes):
        if dt in (pl.Float32, pl.Float64):
            count_inf = df.select(
                (pl.col(c).is_infinite()).sum()
            ).item()
            if count_inf > 0:
                inf_dict[c] = count_inf

    print(f"Columnas con inf o -inf encontradas: {len(inf_dict)}")
    for col, cnt in inf_dict.items():
        print(f"   {col}: {cnt:,} valores")

    return inf_dict
detectar_inf_polars(df_pl)


Columnas con inf o -inf encontradas: 5
   mpayroll_sobre_edad: 34 valores
   mpayroll_sobre_edad_lag1: 34 valores
   mpayroll_sobre_edad_lag2: 34 valores
   mpayroll_sobre_edad_delta1: 34 valores
   mpayroll_sobre_edad_delta2: 34 valores


{'mpayroll_sobre_edad': 34,
 'mpayroll_sobre_edad_lag1': 34,
 'mpayroll_sobre_edad_lag2': 34,
 'mpayroll_sobre_edad_delta1': 34,
 'mpayroll_sobre_edad_delta2': 34}

# ENN 

In [ ]:
if __name__ == "__main__":
    import time
    from datetime import datetime
    from dateutil.relativedelta import relativedelta
    import os  # para crear carpetas

    # ============================================================
    # CONFIGURACION GENERAL
    # ============================================================
    SEED_UNDERSAMPLE = 276779
    UNDERSAMPLING = 0.05
    MULTI_SEED = True
    MEJOR_CANARITOS = 5
    MEJOR_GRADIENT_BOUND = 0.4

    K_NEIGHBORS_ENN = 7
    USE_ENN = True

    TRAIN_HASTA_MONTH = 202106
    FUTURE_MONTH      = 202108


    print("="*70)
    print("🚀 EXPERIMENTO FINAL CON ENSAMBLE + ENN(k=7)")
    print("="*70)

    tiempo_inicio_total = time.time()

    # ============================================================
    # PASO 1: PROCESAR DATASET BASE
    # ============================================================
    print("📂 Procesando dataset base...")
    df, meses = procesamiento_general(df_pl)

    # ============================================================
    # PASO 2: CALCULAR MESES DE TRAIN
    # ============================================================
    TRAIN_MONTHS = [m for m in meses if 201901 <= m <= TRAIN_HASTA_MONTH]
    TRAIN_MONTHS = sorted(TRAIN_MONTHS)

    print(f"\n📅 Meses de entrenamiento: {TRAIN_MONTHS}")
    print(f"   FUTURE: {FUTURE_MONTH}")

    # ============================================================
    # PASO 3: UNDERSAMPLING + ENN(k=7)
    # ============================================================
    print("\n" + "="*70)
    print("PREPARACIÓN DE DATOS (UNDERSAMPLING + ENN)")
    print("="*70)

    # Guardar dataset completo
    dataset_completo = df.clone()

    # Undersampling CONTINUA
    print("\n3B. Aplicando undersampling...")
    dataset_proc = undersampling_experimento(
        df, UNDERSAMPLING, TRAIN_MONTHS, SEED_UNDERSAMPLE
    )

    # ENN (solo con columnas limpias)
    if USE_ENN:
        print("\n3C. Aplicando ENN(k=7)...")

        # --- columnas problemáticas detectadas ----
        cols_inf = [
            'mpayroll_sobre_edad',
            'mpayroll_sobre_edad_lag1',
            'mpayroll_sobre_edad_lag2',
            'mpayroll_sobre_edad_delta1',
            'mpayroll_sobre_edad_delta2'
        ]

        # Dataset temporal limpio SOLO para ENN
        print("   → Eliminando columnas con inf SOLO para ENN...")
        dataset_proc_for_enn = dataset_proc.drop(cols_inf, strict=False)

        # Ejecutar ENN
        dataset_proc_clean = enn_polars(
            dataset_proc_for_enn,
            k_neighbors=K_NEIGHBORS_ENN,
            n_jobs=-1
        )

        # Recuperar enn_keep y adjuntarlo al dataset_proc original
        dataset_proc = dataset_proc.with_columns(
            dataset_proc_clean['enn_keep']
        )

    else:
        print("\n3C. ENN desactivado. Se usa undersampling solamente.")
        dataset_proc = dataset_proc.with_columns(
            pl.col("undersampling").alias("enn_keep")
        )

    # ============================================================
    # PASO 4: SEPARAR TRAIN Y FUTURE
    # ============================================================
    print("\n4. Separando train y future...")

    dataset_train = dataset_proc.filter(
        (pl.col("foto_mes").is_in(TRAIN_MONTHS)) &
        (pl.col("enn_keep") == 1)
    )

    dataset_future = dataset_completo.filter(
        pl.col("foto_mes") == FUTURE_MONTH
    )

    print(f"   Train final:  {len(dataset_train):,} registros")
    print(f"   Future final: {len(dataset_future):,} registros")

    # ============================================================
    # PASO 5: CREAR TARGETS
    # ============================================================
    print("\n5. Creando targets...")

    dataset_train = dataset_train.with_columns(
        (pl.col("clase_ternaria").is_in(["BAJA+1", "BAJA+2"]))
        .cast(pl.Int8)
        .alias("target")
    )

    dataset_future = dataset_future.with_columns(
        (pl.col("clase_ternaria") == "BAJA+2")
        .cast(pl.Int8)
        .alias("target_baja2")
    )

    # ============================================================
    # PASO 6: ARMAR MATRICES NUMPY
    # ============================================================
    print("\n6. Preparando matrices numpy...")

    exclude_cols = [
        'clase_ternaria', 'target', 'target_baja2',
        'foto_mes', 'numero_de_cliente',
        'periodo0', 'periodo1', 'periodo2',
        'azar', 'undersampling', 'clase_peso',
        'mprestamos_personales', 'cprestamos_personales',
        'enn_keep'
    ]

    feature_cols = [c for c in dataset_train.columns if c not in exclude_cols]

    print(f"   N° features: {len(feature_cols)}")

    X_train = dataset_train.select(feature_cols).to_numpy()
    y_train = dataset_train['target'].to_numpy()

    X_future = dataset_future.select(feature_cols).to_numpy()
    y_future = dataset_future['target_baja2'].to_numpy()

    print(f"   X_train shape:  {X_train.shape}")
    print(f"   X_future shape: {X_future.shape}")

    # ============================================================
    # PASO 7: ENTRENAR MODELOS LGBM
    # ============================================================
    print("\n" + "="*70)
    print("🚀 ENTRENANDO LGBM + ENSEMBLE")
    print("="*70)

    modelos, ganancia_prom_ind, n_envios_prom_ind = entrenar_zlightgbm(
        X_train, y_train,
        X_future, y_future,
        canaritos=MEJOR_CANARITOS,
        gradient_bound=MEJOR_GRADIENT_BOUND,
        multi_seed=MULTI_SEED
    )

    # ============================================================
    # NUEVO: GUARDAR MODELOS INDIVIDUALES
    # ============================================================
    print("\n" + "="*70)
    print("💾 GUARDANDO MODELOS INDIVIDUALES")
    print("="*70)
    
    # Crear carpeta para los modelos si no existe
    models_dir = "./modelos_finales"
    os.makedirs(models_dir, exist_ok=True)
    
    # Guardar cada modelo
    for i, modelo in enumerate(modelos, 1):
        # Opción 1: Formato texto (.txt) - MÁS LEGIBLE
        model_path_txt = f"{models_dir}/modelo_seed_{i:02d}.txt"
        modelo.save_model(model_path_txt)
        print(f"   ✅ Guardado: {model_path_txt}")
        
    
    print(f"\n✅ {len(modelos)} modelos guardados en: {models_dir}/")
    
    # Guardar tambien informacion del ensemble
    import json
    metadata = {
        "n_modelos": len(modelos),
        "train_hasta_month": TRAIN_HASTA_MONTH,
        "future_month": FUTURE_MONTH,
        "n_features": len(feature_cols),
        "feature_names": feature_cols,
        "ganancia_promedio_individual": float(ganancia_prom_ind),
        "n_envios_promedio_individual": float(n_envios_prom_ind),
        "semillas": [
               811123, 472457, 670303, 531919, 573329,
                    722167, 429217, 421657, 354779, 998287 
        ],
        "hiperparametros": {
            "canaritos": MEJOR_CANARITOS,
            "gradient_bound": MEJOR_GRADIENT_BOUND,
            "undersampling": UNDERSAMPLING,
            "use_enn": USE_ENN,
            "k_neighbors_enn": K_NEIGHBORS_ENN
        }
    }
    
    metadata_path = f"{models_dir}/metadata_ensemble.json"
    with open(metadata_path, 'w') as f:
        json.dump(metadata, indent=2, fp=f)
    print(f"✅ Metadata guardado en: {metadata_path}")

    # ============================================================
    # PASO 8: ENSAMBLE
    # ============================================================
    print("\n" + "="*70)
    print("📊 CALCULANDO PREDICCIONES DEL ENSEMBLE")
    print("="*70)
    
    preds_future = [m.predict(X_future, num_iteration=m.best_iteration)
                    for m in modelos]

    preds_matrix = np.vstack(preds_future)
    y_pred_ensamble = preds_matrix.mean(axis=0)

    ganancia_ens, n_envios_ens = calcular_ganancia_optima(y_future, y_pred_ensamble)

    print(f"\n✅ Ganancia ENSEMBLE: ${ganancia_ens:,.0f}")
    print(f"   Número óptimo de envíos: {n_envios_ens}")

    # ============================================================
    # FIN
    # ============================================================
    tiempo_fin_total = time.time()
    tiempo_total = tiempo_fin_total - tiempo_inicio_total
    
    print("\n" + "="*70)
    print("🏁 EXPERIMENTO COMPLETADO")
    print("="*70)
    print(f"⏱️  Tiempo total: {tiempo_total/60:.2f} minutos")
    print(f"💾 Modelos guardados: {len(modelos)}")
    print(f"📊 Ganancia ensemble: ${ganancia_ens:,.0f}")
    print("="*70)

🚀 EXPERIMENTO FINAL CON ENSAMBLE + ENN(k=7)
📂 Procesando dataset base...
Dataset featureado recibido desde memoria (R -> Python -> Polars)
Filas: 4,717,958  |  Columnas: 1386
Meses en el dataset: [201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004, 202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108]
Generando cant_nulos...
cant_nulos OK
Generando aguinaldo...
aguinaldo OK
Generando target...
Target creado

📅 Meses de entrenamiento: [201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004, 202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106]
   FUTURE: 202108

PREPARACIÓN DE DATOS (UNDERSAMPLING + ENN)

3B. Aplicando undersampling...

3C. Aplicando ENN(k=7)...
   → Eliminando columnas con inf SOLO para ENN...


In [ ]:
clientes_future = dataset_future.select("numero_de_cliente").to_numpy().flatten()
print(f"✅ Clientes recuperados: {len(clientes_future):,}")



print("\n" + "="*70)
print("💾 GENERANDO CSV FINAL CON PREDICCIONES")
print("="*70)

# Crear DataFrame con las predicciones
df_predicciones = pl.DataFrame({
    "numero_de_cliente": clientes_future,
    "prob_baja": y_pred_ensamble
})

# Ordenar por probabilidad descendente
df_predicciones = df_predicciones.sort("prob_baja", descending=True)

# Determinar numero de envios a usar
n_envios_final = 11000 # Usar el del ensemble
print(f"\n✅ Usando {n_envios_final} envíos")

# Asignar 1 a los top n_envios_final clientes, 0 al resto
df_predicciones = df_predicciones.with_columns([
    pl.when(pl.col("prob_baja").rank("ordinal", descending=True) <= n_envios_final)
    .then(1)
    .otherwise(0)
    .alias("Predicted")
])

# Crear el archivo final solo con numero_de_cliente y Predicted
df_final = df_predicciones.select(["numero_de_cliente", "Predicted"])

# Guardar
output_path = "./predicciones_202008.csv"
df_final.write_csv(output_path)

print(f"\n✅ Archivo guardado: {output_path}")
print(f"   Total clientes: {len(df_final):,}")
print(f"   Predichos como BAJA (1): {df_final['Predicted'].sum():,}")
print(f"   Predichos como NO BAJA (0): {(df_final['Predicted'] == 0).sum():,}")

# También guardar versión con probabilidades para análisis
output_path_prob = "./predicciones_202008_con_prob.csv"
df_predicciones.write_csv(output_path_prob)
print(f"\n📊 Archivo con probabilidades guardado: {output_path_prob}")

print("\n" + "="*70)
print("🏁 CSV GENERADO EXITOSAMENTE")
print("="*70)

✅ Clientes recuperados: 164,822

💾 GENERANDO CSV FINAL CON PREDICCIONES

✅ Usando 11000 envíos

✅ Archivo guardado: ./predicciones_202008.csv
   Total clientes: 164,822
   Predichos como BAJA (1): 11,000
   Predichos como NO BAJA (0): 153,822

📊 Archivo con probabilidades guardado: ./predicciones_202008_con_prob.csv

🏁 CSV GENERADO EXITOSAMENTE
